In [4]:
import pandas as pd
import numpy as np
import datetime as dt
#from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns

from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.linear_model import CoxPHSurvivalAnalysis
from lifelines import CoxPHFitter

from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import FitFailedWarning
import sksurv.util
import warnings

from sksurv.preprocessing import OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statsmodels.api as sm
import seaborn as sns
from sklearn.metrics import classification_report

from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter
from lifelines.statistics import proportional_hazard_test

import simpy
import random
import statistics


In [5]:
df=pd.read_csv("../data/preprocessed_data.csv") #import data

#we are only using data past 2017, pre-2017 data is used to construct donor history variables
mask = (df['Visit_yr'] >= 2017) 
df = df.loc[mask]

# craete a column for outcome type during next visit
df["fu_outcome"] = np.where(df['DonorID'] == df['DonorID'].shift(-1), df['OUTCOME_TYPE'].shift(-1), 10)

#drop other deferrals
df.drop(df[(df['OUTCOME_TYPE']== 'other deferral')].index, inplace=True)

# outcome is 1- if donor recieves hgb deferral on follow up visit
df['hgb_def_fu']=np.where(df['fu_outcome'] == 'low hgb', 1, 0)




C:\Users\jagir\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
#keep only wb and drbc or rbc
wb=['WB', "DEF PROD", 'WBSB', 'WBMED', 'RBCAPH']
df= df.loc[(df['donation_product_x'] == 'WB') | (df['donation_product_x']== 'RBCAPH')|(df['donation_product_x'] == 'DEF PROD')]
df['donation_product_x'].value_counts()

WB          3652678
DEF PROD     203874
RBCAPH         5882
Name: donation_product_x, dtype: int64

In [7]:
#for cox fitter
df_cox2= df.loc[:,['time_to_return', 'CENSORED','Visit_Age',  'first_time' , 'Fixed_mobile' ,'sex', 'donation_product_x' ,'race', 'OUTCOME_TYPE', 
                 'cum_lifetime_donations', 'unit_rbc_loss', 'rbc_loss_last_12_months', 
                 'rbc_loss_last_24_months', 'days_since_last_rbc_loss', 'days_since_last_drbc_loss']]


X3=pd.get_dummies(df_cox2)
X3.fillna(0, inplace=True)
# add interactions
X3['first_time*fixed']=X3["first_time"]*X3['Fixed_mobile']
X3['first_time*hgb']=X3["first_time"]*X3['OUTCOME_TYPE_low hgb']
X3['hgb*fixed']=X3["OUTCOME_TYPE_low hgb"]*X3['Fixed_mobile']


In [8]:
# set up for logistic regression
#get dummies of categorical variables 
df_log= df.loc[:,['time_to_return','Visit_Age',  'first_time' , 'Fixed_mobile' ,'sex', 'donation_product_x' ,'race', 'OUTCOME_TYPE', 
                 'cum_lifetime_donations', 'unit_rbc_loss', 'rbc_loss_last_12_months', 
                 'rbc_loss_last_24_months', 'days_since_last_rbc_loss', 'days_since_last_drbc_loss']]
                 
                 #'first_time*fixed', 'first_time*hgb', 'hgb*fixed']]

X2=pd.get_dummies(df_log)
X2.fillna(0, inplace=True)
# add interactions
X2['first_time*fixed']=X2["first_time"]*X2['Fixed_mobile']
X2['first_time*hgb']=X2["first_time"]*X2['OUTCOME_TYPE_low hgb']
X2['hgb*fixed']=X2["OUTCOME_TYPE_low hgb"]*X2['Fixed_mobile']

Y2= df.loc[:,['hgb_def_fu']]
X2.head()

,time_to_return,Visit_Age,first_time,Fixed_mobile,cum_lifetime_donations,unit_rbc_loss,rbc_loss_last_12_months,rbc_loss_last_24_months,days_since_last_rbc_loss,days_since_last_drbc_loss,...,race_African Black,race_Asian,race_Mixed Race,race_White,race_unknown,OUTCOME_TYPE_completed,OUTCOME_TYPE_low hgb,first_time*fixed,first_time*hgb,hgb*fixed
4,379.0,40.9,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,1,0,0,0,0
5,72.0,41.9,0,1,23.0,1.0,1.0,2.0,379.0,3650.0,...,0,0,0,1,0,1,0,0,0,0
6,68.0,42.1,0,1,24.0,1.0,2.0,3.0,72.0,3650.0,...,0,0,0,1,0,1,0,0,0,0
7,67.0,42.3,0,1,25.0,1.0,3.0,4.0,68.0,3650.0,...,0,0,0,1,0,1,0,0,0,0
8,59.0,42.5,0,1,26.0,1.0,4.0,5.0,67.0,3650.0,...,0,0,0,1,0,1,0,0,0,0


In [9]:
#train model
X_train3, X_test3 = train_test_split(X3, test_size=0.9, random_state=0)
cph = CoxPHFitter(penalizer=0.0001)
cph.fit(X_train3, duration_col = 'time_to_return', event_col = 'CENSORED')
cph.print_summary()

cph.score(X_test3)


<lifelines.CoxPHFitter: fitted with 386243 total observations, 54828 right-censored observations>
             duration col = 'time_to_return'
                event col = 'CENSORED'
                penalizer = 0.0001
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 386243
number of events observed = 331415
   partial log-likelihood = -3906351.09
         time fit was run = 2023-04-13 13:01:57 UTC

---
                              coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                           
Visit_Age                     0.00       1.00       0.00             0.00             0.00                 1.00                 1.00
first_time                   -0.22       0.80       0.01            -0.23            -0.20                 0.79                 0.82
Fixed_mobile                  0.22       1.25       0.00             0.22             0.23                 1.24                 1.26
cum_lifetime_donations        0.00       1.00       0.00             0.00             0.00                 1.00                 1.00
unit_rbc_loss                -0.30       0.74       0.05            -0.39            -0.21                 0.68                 0.81
rbc_loss_last_12_months       0.14       1.15       0.00             0.14             0.15                 1.15                 1.16
rbc_loss_last_24_months       0.06       1.06       0.00             0.06             0.06                 1.06                 1.07
days_since_last_rbc_loss      0.00       1.00       0.00             0.00             0.00                 1.00                 1.00
days_since_last_drbc_loss     0.00       1.00       0.00             0.00             0.00                 1.00                 1.00
sex_F                        -0.01       0.99       0.23            -0.45             0.44                 0.64                 1.55
sex_M                         0.01       1.01       0.23            -0.44             0.45                 0.64                 1.57
donation_product_x_DEF PROD  -0.04       0.96       0.62            -1.25             1.18                 0.29                 3.24
donation_product_x_RBCAPH     0.02       1.02       0.61            -1.17             1.21                 0.31                 3.36
donation_product_x_WB         0.04       1.04       0.61            -1.15             1.23                 0.32                 3.41
race_African Black            0.03       1.03       0.21            -0.37             0.43                 0.69                 1.54
race_Asian                   -0.12       0.89       0.21            -0.52             0.29                 0.59                 1.34
race_Mixed Race               0.04       1.04       0.21            -0.36             0.45                 0.69                 1.56
race_White                   -0.00       1.00       0.21            -0.41             0.40                 0.66                 1.49
race_unknown                 -0.09       0.92       0.21            -0.49             0.32                 0.61                 1.37
OUTCOME_TYPE_completed        0.04       1.04       0.62            -1.18             1.25                 0.31                 3.50
OUTCOME_TYPE_low hgb         -0.04       0.96       0.62            -1.25             1.18                 0.29                 3.24
first_time*fixed              0.09       1.10       0.02             0.06             0.12                 1.06                 1.13
first_time*hgb               -0.01       0.99       0.03            -0.07             0.06                 0.93                 1.06
hgb*fixed                     0.01       1.01       0.02            -0.02             0.04                 0.98                 1.04

                              cmp to      z      p   -log2(p)
covariate                  

-12.000067994773087

In [29]:
#logistic regression to predict hgb deferral on follow up visit

#train model
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, Y2, test_size=0.7, random_state=0)

#print summary
model = sm.Logit(y_train2, X_train2)
result = model.fit()
print(result.summary())

#train model
logreg = LogisticRegressionCV(cv=5, random_state=0)
logreg.fit(X_train2.values, y_train2.values)

#make predictions

y_pred = logreg.predict(X_test2)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test2, y_test2)))
print(classification_report(y_test2, y_pred))


Optimization terminated successfully.
         Current function value: 0.165342
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:             hgb_def_fu   No. Observations:              1158730
Model:                          Logit   Df Residuals:                  1158709
Method:                           MLE   Df Model:                           20
Date:                Thu, 13 Apr 2023   Pseudo R-squ.:                  0.1046
Time:                        09:31:03   Log-Likelihood:            -1.9159e+05
converged:                       True   LL-Null:                   -2.1397e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
time_to_return                 -0.0040   5.52e-05    -72.098      0.000   

C:\Users\jagir\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jagir\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\jagir\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (m

Accuracy of logistic regression classifier on test set: 0.95


C:\Users\jagir\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jagir\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      1.00      0.98   2581162
           1       0.00      0.00      0.00    122542

    accuracy                           0.95   2703704
   macro avg       0.48      0.50      0.49   2703704
weighted avg       0.91      0.95      0.93   2703704



C:\Users\jagir\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
#SIMULATION MODEL


def simulate_donation(cph, logistic_model, idi, donor):
    donor=donor.to_frame()
    donor=donor.T 
    
    donor_for_cph=donor.drop(labels=['time_to_return','CENSORED'], axis=1)
    # Use the Cox model to predict the time to return
    time_to_return = cph.predict_median(donor)

    # Simulate whether the donor returns based on the predicted time
    if np.random.uniform(0, 1) > 1 - np.exp(-time_to_return):
        return 0
    
    if (time_to_return < idi) and donor['OUTCOME_TYPE_completed'].any()==1:
        return 0
    
    
    donor['time_to_return']=time_to_return
    X=donor.drop(labels=['CENSORED'], axis=1)
    # Use the logistic model to predict the probability of hemoglobin deferral
    hemoglobin_prob = logistic_model.predict(X.values)

    # Simulate whether the donor is deferred based on the predicted probability
    if np.random.uniform(0, 1) < hemoglobin_prob:
        return 1

    # If the donor is not deferred, record the donation and the inter-donation interval
    return 2


In [25]:
#a function to calculate comparative effectiveness and estimated revenue

def compute_results(DONOR_COUNT, successful_donations, deferrals):
    cost_of_def=211.83
    cost_of_sd= 609.28
    profit_from_sd=2291.30
    # Compute comparative effectiveness 
    print("Total donors: {}".format(DONOR_COUNT))
    print("Successful donations: {}".format(successful_donations))
    print("Deferrals: {}".format(deferrals))
    print("Deferral rate: {:.2f}%".format(deferrals/DONOR_COUNT*100))

    # compute estimated revenue
    #Estimated revenue = profit from completed donation - cost of a hemoglobin deferral or completed donation
    estimated_rev= successful_donations*profit_from_sd - (cost_of_def*deferrals + cost_of_sd*successful_donations)
    print("Esimated Revenue: {}".format(round(estimated_rev,2)))
    

In [26]:
#sample from data 10,000 times
donors = 10000

df_sim=X3.sample(donors, replace=True, random_state=9)

df_sim.head()


,time_to_return,CENSORED,Visit_Age,first_time,Fixed_mobile,cum_lifetime_donations,unit_rbc_loss,rbc_loss_last_12_months,rbc_loss_last_24_months,days_since_last_rbc_loss,...,race_African Black,race_Asian,race_Mixed Race,race_White,race_unknown,OUTCOME_TYPE_completed,OUTCOME_TYPE_low hgb,first_time*fixed,first_time*hgb,hgb*fixed
3701151,192.0,1,38.4,0,1,9.0,1.0,4.0,6.0,85.0,...,1,0,0,0,0,1,0,0,0,0
1222538,125.0,1,44.2,0,0,9.0,1.0,4.0,5.0,64.0,...,0,0,0,1,0,1,0,0,0,0
5089886,31.0,0,30.5,1,0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,1,0,0,0,0
644884,56.0,1,50.6,0,1,115.0,1.0,6.0,11.0,77.0,...,0,0,0,1,0,1,0,0,0,0
3931484,62.0,1,29.4,0,0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,1,0,0,0,0


In [27]:
#RUN SIMULATION

#initialize variables
#for standard 8 week for boths
# deferrals_standard = 0
successful_donations_standard = 0
deferrals_standard=0
# 12 for women, 8 for men
deferrals_alt2 = 0
successful_donations_alt2 = 0

# 12 for both
deferrals_alt3 = 0
successful_donations_alt3 = 0

count=0

# run simulation
for index, donor in df_sim.iterrows():
    
    #standard
    std=simulate_donation(cph, logreg, 57, donor)
    if std==2:
        successful_donations_standard+=1
        
    else:
        deferrals_standard +=1
        
#for index, donor in df_sim.iterrows():     
    #alt2
    if donor['sex_M'] == 1:
        alt2=simulate_donation(cph, logreg, 56, donor)
    else:
        alt2=simulate_donation(cph, logreg, 84, donor)
        
    if alt2==2:
        successful_donations_alt2 +=1
       
    else:
        deferrals_alt2+=1
        
        
#for index, donor in df_sim.iterrows():    
    #alt3
    alt3=simulate_donation(cph, logreg, 84, donor)
    if alt3==2:
        successful_donations_alt3 +=1
        
    else:
        deferrals_alt3 +=1
        
        
        

In [30]:
   #compute results for standard
print("Standard Inter-donation Interval of 56 days for both men and women")
compute_results(donors, successful_donations_standard,deferrals_standard)
    
    #compute results for alt2
print("Alternative Inter-donation Interval of 56 days for men and 84 days for women")
compute_results(donors, successful_donations_alt2,deferrals_alt2)
    
    #compute results from alt3
print("Alternative Inter-donation Interval of 84 days for both men and women")
compute_results(donors, successful_donations_alt3,deferrals_alt3)

Standard Inter-donation Interval of 56 days for both men and women
Total donors: 10000
Successful donations: 10000
Deferrals: 0
Deferral rate: 0.00%
Esimated Revenue: 16820200.0
Alternative Inter-donation Interval of 56 days for men and 84 days for women
Total donors: 10000
Successful donations: 8893
Deferrals: 1107
Deferral rate: 11.07%
Esimated Revenue: 14723708.05
Alternative Inter-donation Interval of 84 days for both men and women
Total donors: 10000
Successful donations: 6643
Deferrals: 3357
Deferral rate: 33.57%
Esimated Revenue: 10462545.55
